In [1]:
print("hello world")

hello world


In [10]:

import pandas as pd
import numpy as np
import os


non_value_columns = np.array([
    {"name": 'DT_ANO_LETIVO_INICIO'},
    {"name": 'DT_ANO_LETIVO_TERMINO'},
    {"name": 'NO_ENTIDADE'},
    {"name": 'NO_MUNICIPIO'},
    {"name": 'SG_UF'},
    {"name": 'NO_REGIAO'},
    {"name": 'NU_ANO_CENSO'},   
    {"name": "DS_ENDERECO"},
    {"name": "NU_ENDERECO"},
    {"name": "DS_COMPLEMENTO"},
    {"name": "NO_BAIRRO"},
    {"name": "NU_DDD"},
    {"name": "NU_TELEFONE"},
    {"name": "TP_DEPENDENCIA"},
    {"name":"TP_SITUACAO_FUNCIONAMENTO"},
    { "name":"TP_CATEGORIA_ESCOLA_PRIVADA"},
    { "name":"TP_LOCALIZACAO"},
    {"name":"IN_COZINHA"},
    {"name":"IN_ESGOTO_INEXISTENTE"},
    {"name":"IN_ENERGIA_INEXISTENTE"},
    {"name":"IN_AREA_VERDE"},
    {"name":"IN_BANHEIRO"},
    {"name":"IN_DESPENSA"},
    {"name":"IN_SECRETARIA"},
    {"name":"IN_TRATAMENTO_LIXO_INEXISTENTE"},
    {"name":"IN_REFEITORIO"},
    {"name":"TP_DEPENDENCIA"},
    {"name":"IN_BIBLIOTECA"}
])
columns = [x['name'] for x in non_value_columns]




df_school_census_final = pd.read_csv("data/microdados_ed_basica_2022/dados/microdados_ed_basica_2022.csv",encoding="ISO-8859-1", sep = ';')
df_school_census_final = df_school_census_final[columns]

for year in range(2012,2022):
    csv_file = f'data/microdados_ed_basica_{year}/dados/microdados_ed_basica_{year}.csv'
    df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')
    df_current = df_current[columns]
    df_school_census_final = pd.concat([df_school_census_final,df_current])
    
#df_school_census_final.to_csv(r'C:\Users\Matheus Rafalski\Downloads\export_dataframe.csv', index=False)


#df_school_census_columns = pd.read_csv("Downloads/export_dataframe.csv",encoding="utf8", sep = ',')
df_school_census_columns = df_school_census_final

columns = np.array([
    {"name": 'IN_COZINHA', "1": 80, "0":0},
    {"name": 'IN_BIBLIOTECA', "1": 100, "0":0},
    {"name": 'IN_ESGOTO_INEXISTENTE', "1": 0, "0":100},
    {"name": 'IN_ENERGIA_INEXISTENTE', "1": 0, "0":100},
    {"name": 'IN_AREA_VERDE', "1": 80, "0":0},
    {"name": 'IN_BANHEIRO', "1": 100, "0":0},
    {"name": 'IN_DESPENSA', "1": 80, "0":0},
    {"name": 'IN_SECRETARIA', "1": 100, "0":0},
    {"name": 'IN_TRATAMENTO_LIXO_INEXISTENTE', "1": 0, "0":80},
    {"name": 'IN_REFEITORIO', "1": 80, "0":0},
])

result = np.concatenate((columns, non_value_columns))


df_school_census_columns.dropna(axis=0, subset=['DT_ANO_LETIVO_INICIO'], inplace=True)
df_school_census_columns = df_school_census_columns.replace('',np.nan)
df_school_census_columns = df_school_census_columns[map(lambda column: column['name'], non_value_columns)]
df_school_census_columns = df_school_census_columns[df_school_census_columns["DT_ANO_LETIVO_INICIO"] != "0"]


df_school_census_columns = df_school_census_columns.fillna(value=df_school_census_columns.mode().iloc[0])
# Iterando sobre as colunas do dataframe
for column in columns:
    col = column['name']
    df_school_census_columns[col] = df_school_census_columns[col].astype(int)

def transform_situacao_funcionamento(x):
    if x == 1:
        return "Ativa"
    elif x == 2:
        return "Paralisada"
    else: 
        return "Extinta"
    

df_school_census_columns["TP_SITUACAO_FUNCIONAMENTO"] = df_school_census_columns["TP_SITUACAO_FUNCIONAMENTO"].apply(transform_situacao_funcionamento)
def transform_localizacao(x):
    if x == 1:
        return "Urbana"
    elif x == 2:
        return "Rural"
    

df_school_census_columns["TP_LOCALIZACAO"] = df_school_census_columns["TP_LOCALIZACAO"].apply(transform_localizacao)

def transform_categoria_escola(x):
    if x == 1:
        return "Particular"
    elif x == 2:
        return "Comunitária"
    elif x == 3:
        return "Confessional"
    elif x == 4:
        return "Filantrópica"
    else:
        return "Pública"

df_school_census_columns["TP_CATEGORIA_ESCOLA_PRIVADA"] = df_school_census_columns["TP_CATEGORIA_ESCOLA_PRIVADA"].apply(transform_categoria_escola)

def transform_dependencia(x):
    if x.eq(1).any():
        return "Federal"
    elif x.eq(2).any():
        return "Estadual"
    elif x.eq(3).any():
        return "Municipal"
    elif x.eq(4).any():
        return "Privada"
    else:
        return x

df_school_census_columns["TP_DEPENDENCIA"] = df_school_census_columns["TP_DEPENDENCIA"].apply(transform_dependencia)




def transform(x,column):
    x_object = column
    
    if x == 1:
        return  x_object['1']
    elif x == 0:
        return  x_object['0']
    else:
        return x

# apply the function to each column using apply()

for i in columns:
    df_school_census_columns[i['name']] = df_school_census_columns[i['name']].apply(transform, column=i)

print("Dataframe resultante:")
columns_name= [x['name'] for x in columns]

estrutural_columns = df_school_census_columns[map(lambda column: column, columns_name)]
estrutural_columns = estrutural_columns.assign(total=estrutural_columns.sum(axis=1))
df_school_census_columns['total'] = estrutural_columns['total']
df_school_census_columns.drop(columns_name, axis=1, inplace=True)

# merge pib csv in df_school_census and then removce unused columns and apply etl

df_school_census_columns = df_school_census_columns.drop(columns=['Código do Município', 'Região Metropolitana','Valor Agropecuária',
                      'Valor Indústria','Valor Serviços','Valor Administração','Valor Total Bruto'
                      ,'Impostos Liquidos','Valor Total','Atividade com maior valor adicionado bruto','Ano do Cesno'])

df_school_census_columns.columns.values[1] = "Região"
df_school_census_columns.columns.values[2] = "Sigla"
df_school_census_columns.columns.values[3] = "Estado"
df_school_census_columns.columns.values[4] = "Município"
df_school_census_columns.columns.values[7] = "Escola"
df_school_census_columns.columns.values[8] = "Categoria"

def create_structure(x):
    if x >= 800:
        return "Ótima"
    elif (x >= 600) & (x < 800):
         return "Boa"
    elif (x >= 400) & (x < 600):
        return "Média"
    elif (x >= 200) & (x < 400):
        return "Ruim"
    elif (x >= 0) & (x < 200):
        return "Precária"
    
df_school_census_columns["Estrutura"] = df_school_census_columns["total"].apply(createStructure)




#df_school_census_columns = df_school_census_columns.dropna(axis=0, subset=['DT_ANO_LETIVO_INICIO'])

C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_14396\3332641533.py:42: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_school_census_final = pd.read_csv("data/microdados_ed_basica_2022/dados/microdados_ed_basica_2022.csv",encoding="ISO-8859-1", sep = ';')


2012


C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_14396\3332641533.py:48: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')


2013


C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_14396\3332641533.py:48: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')


...

C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_14396\3332641533.py:48: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')


2020


C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_14396\3332641533.py:48: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')


2021


C:\Users\Matheus Rafalski\AppData\Local\Temp\ipykernel_14396\3332641533.py:48: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_current = pd.read_csv(csv_file,encoding="ISO-8859-1", sep = ';')


In [2]:
# import pandas as pd
# df_school_census_columns = pd.read_csv("Downloads/export_dataframe.csv",encoding="utf8", sep = ',')

# df_school_census_columns.to_csv(r'C:\Users\Matheus Rafalski\Downloads\csv_escolas.csv', index=False)
# df_school_census_columns

NameError: name 'non_value_columns' is not defined

In [32]:
df_school_census_columns.drop(a, axis=1, inplace=True)

In [61]:
df_school_census_columns = df_school_census_columns.sort_values(by='total',ascending=False)
